In [ ]:
load "/work/pannote/panbot_note.rb"
load "/work/panbot/panbot.rb"

class PayoutBot < PanBot
    attr_accessor :config
    
    def mainLoop
        if isLastBetable then
            log "#{getCurrentEpoch} #{getCurrentBlock} #{isLastBetable} #{getCurrentPayout} #{getCurrentAmount}"
            payout_bull, payout_bear = getCurrentPayout
            amount = getCurrentAmount

            if [payout_bull, payout_bear].max > @config[:min_payout] and amount >  @config[:min_amount] then
                log "make bet"
                betBull(self,getCurrentAmount * @config[:bet_amount_factor] + @config[:bet_amount_value]) if payout_bull > payout_bear
                betBear(self,getCurrentAmount * @config[:bet_amount_factor] + @config[:bet_amount_value]) if payout_bull < payout_bear
            else
                log "do not bet"
            end
        end
    end

    def initialize(runner, config={})
        @config = config

        super(runner)
    end 
end



:initialize

In [ ]:
min_amount = '20'

"20"

In [ ]:
min_payout = '2.1'

"2.1"

In [ ]:
bet_amount_factor = '0'

"0"

In [ ]:
bet_amount_value = '0.1'

"0.1"

In [ ]:
epoch_begin = '32630'

"32630"

In [ ]:
epoch_end = '41293'

"41293"

In [ ]:
epoch_begin = 32930
epoch_end = 32940

config = {:min_amount => min_amount.to_f, :min_payout=>min_payout.to_f, :bet_amount_factor=>bet_amount_factor.to_f, :bet_amount_value=>bet_amount_value.to_f}

time = Time.now()

runner = SimulationRunner.new()
runner.time_at_epoch(epoch_begin.to_i,epoch_end.to_i)

bot = PayoutBot.new(runner,config)
runner.run

puts "#{Time.now()-time} s"

0.202670876 s


In [ ]:
# calc bot stats


bot.bet_result = bot.bet_result.map { |x| 
    x[:right_bet] = (x[:bet]==(  x[:bull_payout] > x[:bear_payout]  ? "bull" : "bear"  )) ? 1 : 0
    x[:win_bet] = (x[:bet]==x[:bet_result]) ? 1 : 0
    x
}

bot.bet_result = bot.bet_result.map { |x| 
    x[:return_amt] = -1*x[:amount] + ( x[:win_bet]==1 ? (x[:amount] * ( x["#{x[:bet_result]}_payout".to_sym] or 0 ) ) : 0 ) if x[:bet]!=""
    x
}

invest_cnt = bet_cnt = bet_epoch = bot.bet_result.filter {|x| x[:bet]!=""}.count
all_epoch = epoch_end-epoch_begin+1
bet_ratio = bet_epoch/all_epoch.to_f
bet_bull_cnt = bot.bet_result.filter {|x| x[:bet]=="bull"}.count
bet_bull_ratio = bet_bull_cnt/bet_epoch.to_f

avg_bet_amt = bot.bet_result.filter {|x| x[:bet]!=""}.map {|x| x[:amount]}.sum / bet_epoch.to_f
avg_last_block_order = bot.bet_result.filter {|x| x[:bet]!=""}.map {|x| x[:bet_block_order]}.sum / bet_epoch.to_f

right_bet_ratio = bot.bet_result.filter {|x| x[:bet]!=""}.map {|x| x[:right_bet]}.sum / bet_epoch.to_f
win_bet_ratio = bot.bet_result.filter {|x| x[:bet]!=""}.map {|x| x[:win_bet]}.sum / bet_epoch.to_f

invest_amt = bot.bet_result.filter {|x| x[:bet]!=""}.map {|x| x[:amount]}.sum
return_amt = bot.bet_result.filter {|x| x[:bet]!=""}.map {|x| x[:return_amt]}.sum
return_flow = return_amt / all_epoch.to_f

max_retrace = 0
running_return = 0
bot.bet_result.filter {|x| x[:bet]!=""}.map {|x| x[:return_amt]}.each do |x|
    running_return = running_return + x
    max_retrace = running_return if running_return<max_retrace
end


0.07509219414904891

In [ ]:
# puts bot.bet_result
# puts runner.logs.join("\n")

# todo
# 模拟
# Payout策略的
# - 模拟的收益统计，出status
# Fellow策略
# - EpochBet.All- 列出目前为止所有bet
# 模拟的统计，多次随机模拟
# 实盘的runner

In [ ]:
return_flow

0.006826563104458992

In [ ]:
puts bot.bet_result.join("\n")

{:epoch=>32930, :bet=>"bear", :amount=>0.1, :bet_block_order=>1, :bull_payout=>1.8885949036595615, :bear_payout=>2.125372198154222, :bull_amount=>17.154216214955788, :bear_amount=>15.243149104883928, :bet_result=>"bear", :right_bet=>1, :win_bet=>1, :return_amt=>0.1125372198154222}
{:epoch=>32931, :bet=>"bear", :amount=>0.1, :bet_block_order=>1, :bull_payout=>1.9708340777625444, :bear_payout=>2.030042128624773, :bull_amount=>20.628265395107622, :bear_amount=>20.02662301070032, :bet_result=>"bull", :right_bet=>1, :win_bet=>0, :return_amt=>-0.1}
{:epoch=>32932, :bet=>"bull", :amount=>0.1, :bet_block_order=>1, :bull_payout=>2.107348311698147, :bear_payout=>1.903058224260508, :bull_amount=>11.889141986183027, :bear_amount=>13.165421305939326, :bet_result=>"bull", :right_bet=>1, :win_bet=>1, :return_amt=>0.1107348311698147}
{:epoch=>32933, :bet=>"bear", :amount=>0.1, :bet_block_order=>1, :bull_payout=>1.6425903258386487, :bear_payout=>2.556201454939263, :bull_amount=>18.683992081959687, :bea

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>